# VC Signing

The notebook demonstrates signing a VC using the TO BE DEFINED SchnorrSecp256k1 Crypto Suite.


# CryptoSuite Requirements (https://w3c.github.io/vc-data-integrity/#cryptographic-suites)

An example of a data integrity cryptosuite can be found here : https://www.w3.org/TR/vc-di-eddsa/

**The requirements for all data integrity cryptographic suite specifications are as follows:**

1. The specification MUST be published as a human-readable document at a URL.

We need to publish a specification detailing our cryptosuite. I suggest we try to get it into either DIF or the CCG, but we could also just host it ourselves.

2. The specification MUST identify a cryptographic suite type and any parameters that can be used with the suite.

Proposed cryptosuite type: `schnorr-secp256k1-2024`

Note: We may want to define two cryptosuites in a single spec which would vary according to the transformation algorithm used. JSON Canonicalization (jcs) or RDF Canonicalization (rdfc). We could define two cryptosite types: `schnorr-secp256k1-jcs-2024` and `schnorr-secp256k1-rdfc-2024`. This is a common pattern, see https://www.w3.org/TR/vc-di-eddsa/#instantiate-cryptosuite

We need to review what is meant by the parameters used with the suite. I believe it is about how we instantiate the cryptosuite.
 
3. The specification MUST detail the transformation algorithms (if any), parameters, and other necessary details, used to modify input data into the data to be protected.

This is typically the canonicialization step. It might vary if we have multiple canoncalization schemes e.g. (jcs and rdfc). Step 4 in this notebook details the transformation algorithm using the JSON Canonicalization scheme.

4. The specification MUST detail the hashing algorithms parameters, and other necessary details used to perform cryptographic hashing to the data to be protected.

I believe we just have to pick a preffered hashing algorithm. I suggest SHA256. See an example in Step 5 of this notebook.

5. The specification MUST detail the proof serialization algorithms, parameters, and other necessary details used to perform cryptographic protection of the data.

This is about how we will generate the proof bytes: Propose a Schnorr signature following BIP340

6. The specification MUST detail the proof verification algorithms, parameters, and other necessary details used to perform cryptographic verification of the data.


7. The specification MUST define a data integrity cryptographic suite instantiation algorithm that accepts a set of options (map options) and returns a cryptosuite instance (struct cryptosuite). This algorithm SHOULD be listed in the Verifiable Credential Extensions document. A data integrity cryptographic suite instance struct has the following items.


This is an implementation TODO.

**createProof**
- An algorithm that takes an input document (map inputDocument) and proof options (map options) as input, and produces a data integrity proof (map) or an error.

**verifyProof**
- An algorithm that takes a secured data document (map securedDocument) as input, and produces a cryptosuite verification result or an error. The cryptosuite verification result is a struct that contains the following items:
verified
- A boolean that is true if the verification succeeded, or false otherwise.
verifiedDocument
- A map that represents the secured data document with the verified proofs removed if verified is true, or null otherwise.
- The structure MAY contain other implementation-specific information that is useful for developers, such as debugging information. If an error is produced, the verification process failed to complete. An error, such as a network error, does not mean that a future attempt at verification would fail.

8. The specification MUST detail any known resource starvation attack that can occur in an algorithm and provide testable mitigations against each attack.

Not sure.

9. The specification MUST contain a Security Considerations section detailing security considerations specific to the cryptographic suite.

Spec writing TODO

10. The specification MUST contain a Privacy Considerations section detailing privacy considerations specific to the cryptographic suite.

Spec writing TODO

11. The JSON-LD context associated with the cryptographic suite MUST have its terms protected from unsafe redefinition, by use of the @protected keyword.

I actually don't think we need a new context here. We are just using the context terms from Data Integrity

# 0. Initial Setup

We do some initial setup to create a DID and an unsecuredDocument to be signed

## 0.1. Add libbtc1 python library to path

In [44]:
import sys
import os

notebooks_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the Notebooks directory to the sys.path
sys.path.append(notebooks_path)

## 0.2. Create key pair

In [45]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey


In [46]:
## Run this if you want a new hardware key
# mnemonic = secure_mnemonic()

mnemonic = "prosper can dial lumber write coconut express imitate husband isolate inside release brush media please kind comic pill science repeat basic also endorse bronze"
root_hdpriv = HDPrivateKey.from_mnemonic(mnemonic, network="signet")
print("Mnemonic : ", mnemonic)

Mnemonic :  prosper can dial lumber write coconut express imitate husband isolate inside release brush media please kind comic pill science repeat basic also endorse bronze


In [47]:
didkey_purpose = "11"

initial_sk = root_hdpriv.get_private_key(didkey_purpose, address_num=2)
initial_pk = initial_sk.point

print("Secp256k1 PrivateKey", initial_sk.hex())
print("Secp256k1 Public Key", initial_pk.__repr__())

Secp256k1 PrivateKey 73fddc12fc9342bb29ae9b5ed476323bdc693a116e36512455aee830fe8a25a2
Secp256k1 Public Key S256Point(029ad5f6a85d27ee69b133aed273b4f2f5d70ed4a71675019c1a76f04c663526ef)


## 0.3. Deterministically Create DID BTC1 

In [48]:
from libbtc1.did import create_deterministic

did_btc1, did_document = create_deterministic(initial_pk)

In [49]:
did_btc1

'did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65'

In [51]:
import json
print(json.dumps(did_document, indent=2))

{
  "id": "did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65",
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://did-btc1/TBD/context"
  ],
  "verificationMethod": [
    {
      "id": "#initialKey",
      "type": "Multikey",
      "controller": "did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65",
      "publicKeyMultibase": "z66PwJnYvwJLhGrVc8vcuUkKs99sKCzYRM2HQ2gDCGTAStHk"
    }
  ],
  "authentication": [
    "#initialKey"
  ],
  "assertionMethod": [
    "#initialKey"
  ],
  "capabilityInvocation": [
    "#initialKey"
  ],
  "capabilityDelegation": [
    "#initialKey"
  ],
  "service": [
    {
      "id": "#initial_p2pkh",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:1CYzZ6DpweJ5cmhsaFPGqcDmmpAjmnp2eT"
    },
    {
      "id": "#initial_p2wpkh",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:bc1q06m9yn2kxgxg2mara55667958d0a6s2kcz60ph"
    },
    {
      "id": "#initial_p2tr",
      "type

## 0.4. Access Verification Method and Retrive Associated PrivateKey

 Note: a verificationMethod in a DID document provides the public key. A system wishing to create signatures using a verification method according to a cryptosuite MUST be able to retrieve the associated privatekey for the verification method they wish to use. Likely we would want some cryptographic metadata that maps verification methods to derivation paths for a hardware wallet.

In [53]:
verificationMethod = did_document["verificationMethod"][0]

print(json.dumps(verificationMethod, indent=2))

print("Associated PrivateKey : ", initial_sk.wif())

{
  "id": "#initialKey",
  "type": "Multikey",
  "controller": "did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65",
  "publicKeyMultibase": "z66PwJnYvwJLhGrVc8vcuUkKs99sKCzYRM2HQ2gDCGTAStHk"
}
Associated PrivateKey :  cRUB4N3UngnZterPLwe2gVexRXojCYJ5a3KrB16efwZ11CSwBRPt


## 0.5. Define VC to be signed

The VC should include an issuer property referencing the did:btc1 identifier that controls the verificationMethod used for signing

In [54]:
example_vc = {
  "@context": [
    "https://www.w3.org/ns/credentials/v2",
    "https://www.w3.org/ns/credentials/examples/v2"
  ],
  "id": "http://university.example/credentials/58473",
  "type": ["VerifiableCredential", "ExampleAlumniCredential"],
  "validFrom": "2020-01-01T00:00:00Z",
  "credentialSubject": {
    "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
    "alumniOf": {
      "id": "did:example:c276e12ec21ebfeb1f712ebc6f1",
      "name": "Example University"
    }
  }
}

In [55]:
example_vc["issuer"] = did_btc1

In [56]:
unsecured_document = example_vc

# Add Proof (Data Integrity)

https://w3c.github.io/vc-data-integrity/#add-proof

The following algorithm specifies how a digital proof can be added to an input document, and can then be used to verify the output document's authenticity and integrity. Required inputs are an input document (map inputDocument), a cryptosuite instance (struct cryptosuite), and a set of options (map options). Output is a secured data document (map) or an error. Whenever this algorithm encodes strings, it MUST use UTF-8 encoding.

1. Let proof be the result of calling the createProof algorithm specified in cryptosuite.createProof with inputDocument and options passed as a parameters. If the algorithm produces an error, the error MUST be propagated and SHOULD convey the error type.
2. If one or more of the proof.type, proof.verificationMethod, and proof.proofPurpose values is not set, an error MUST be raised and SHOULD convey an error type of PROOF_GENERATION_ERROR.
3. If options has a non-null domain item, it MUST be equal to proof.domain or an error MUST be raised and SHOULD convey an error type of PROOF_GENERATION_ERROR.
4. If options has a non-null challenge item, it MUST be equal to proof.challenge or an error MUST be raised and SHOULD convey an error type of PROOF_GENERATION_ERROR.
5. Let securedDataDocument be a copy of inputDocument.
6. Set securedDataDocument.proof to the value of proof.
8. Return securedDataDocument as the secured data document.


# Create Proof (schnorr-secp256k1-jcs-2024)

The following algorithm specifies how to create a data integrity proof given an unsecured data document. Required inputs are an unsecured data document (map unsecuredDocument), and a set of proof options (map options). A data integrity proof (map), or an error, is produced as output.

1. Let proof be a clone of the proof options, options.
2. If unsecuredDocument.@context is present, set proof.@context to unsecuredDocument.@context.
3. Let proofConfig be the result of running the algorithm in Section XXX Proof Configuration (schnorr-secp256k1-jcs-2024) with proof passed as the proof options parameter.
4. Let transformedData be the result of running the algorithm in Section XXX Transformation (schnorr-secp256k1-jcs-2024) with unsecuredDocument and options passed as parameters.
5. Let hashData be the result of running the algorithm in Section XXX Hashing (schnorr-secp256k1-jcs-2024) with transformedData and proofConfig passed as a parameters.
6. Let proofBytes be the result of running the algorithm in Section XXX Proof Serialization (schnorr-secp256k1-jcs-2024) with hashData and options passed as parameters.
7. Let proof.proofValue be a base58-btc-encoded Multibase value of the proofBytes.
Return proof as the data integrity proof.

## 1. Let proof be a clone of the proof options, options.



In [58]:
import copy
# Input to the createProof algorithm
options = {
    "type": "DataIntegrityProof",
    "cryptosuite": "schnorr-secp256k1-jcs-2024",
    "verificationMethod": f"{did_btc1}{did_document["verificationMethod"][0]["id"]}",
    "proofPurpose": "assertionMethod"
}

In [59]:
proof = copy.deepcopy(options)

## 2. If unsecuredDocument.@context is present, set proof.@context to unsecuredDocument.@context.


In [61]:
if unsecured_document["@context"]:
    proof["@context"] = unsecured_document["@context"]

## 3. Let proofConfig be the result of running the algorithm in Section XXX Proof Configuration (schnorr-secp256k1-jcs-2024) with proof passed as the proof options parameter.


The following algorithm specifies how to generate a proof configuration from a set of proof options that is used as input to the proof hashing algorithm.

The required inputs to this algorithm are proof options (options). The proof options MUST contain a type identifier for the cryptographic suite (type) and MUST contain a cryptosuite identifier (cryptosuite). A proof configuration object is produced as output.

1. Let proofConfig be a clone of the options object.
2. If proofConfig.type is not set to DataIntegrityProof or proofConfig.cryptosuite is not set to schnorr-secp256k1-jcs-2024, an error MUST be raised that SHOULD convey an error type of PROOF_GENERATION_ERROR.
3. If proofConfig.created is set to a value that is not a valid [XMLSCHEMA11-2] datetime, an error MUST be raised and SHOULD convey an error type of PROOF_GENERATION_ERROR.
4. Let canonicalProofConfig be the result of applying the JSON Canonicalization Scheme [RFC8785] to the proofConfig.
Return canonicalProofConfig.

In [63]:
# Input  to the proof configuration algorithm
proofConfigOptions = copy.deepcopy(proof)

### 3.1 Let proofConfig be a clone of the options object.

In [64]:
import copy
proofConfig = copy.deepcopy(proofConfigOptions)

### 3.2. If proofConfig.type is not set to DataIntegrityProof or proofConfig.cryptosuite is not set to schnorr-secp256k1-jcs-2024, an error MUST be raised that SHOULD convey an error type of PROOF_GENERATION_ERROR.

In [65]:
assert proofConfig["type"] == "DataIntegrityProof"
assert proofConfig["cryptosuite"] == "schnorr-secp256k1-jcs-2024"

### 3.3. If proofConfig.created is set to a value that is not a valid [XMLSCHEMA11-2] datetime, an error MUST be raised and SHOULD convey an error type of PROOF_GENERATION_ERROR.


In [66]:
print("TODO: Check datetime if created set")

TODO: Check datetime if created set


### 3.4. Let canonicalProofConfig be the result of applying the JSON Canonicalization Scheme [RFC8785] to the proofConfig.


In [67]:
import jcs
canonicalProofConfig = jcs.canonicalize(proofConfig)

### 3.5 Return canonicalProofConfig

In [68]:
proof_config = canonicalProofConfig
print("Return canonicalProofConfig : ", canonicalProofConfig)

Return canonicalProofConfig :  b'{"@context":["https://www.w3.org/ns/credentials/v2","https://www.w3.org/ns/credentials/examples/v2"],"cryptosuite":"schnorr-secp256k1-jcs-2024","proofPurpose":"assertionMethod","type":"DataIntegrityProof","verificationMethod":"did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65#initialKey"}'


## 4. Let transformedData be the result of running the algorithm in Section XXX Transformation (schnorr-secp256k1-jcs-2024) with unsecuredDocument and options passed as parameters.

Currently this uses the Json Canonicalization Scheme. We intend to shift this to use RDF canonicalization once we have ported Dan's library.

Example spec text: https://www.w3.org/TR/vc-di-eddsa/#transformation-eddsa-jcs-2022


### Spec Text

The following algorithm specifies how to transform an unsecured input document into a transformed document that is ready to be provided as input to the hashing algorithm in Section XXX Hashing (schnorr-secp256k1-jcs-2024).

Required inputs to this algorithm are an unsecured data document (unsecuredDocument) and transformation options (options). The transformation options MUST contain a type identifier for the cryptographic suite (type) and a cryptosuite identifier (cryptosuite). A transformed data document is produced as output. Whenever this algorithm encodes strings, it MUST use UTF-8 encoding.

1. If options.type is not set to the string DataIntegrityProof and options.cryptosuite is not set to the string schnorr-secp256k1-jcs-2024, an error MUST be raised that SHOULD convey an error type of PROOF_VERIFICATION_ERROR.
2. Let canonicalDocument be the result of applying the JSON Canonicalization Scheme [RFC8785] to a JSON serialization of the unsecuredDocument.
3. Return canonicalDocument as the transformed data document.

In [71]:
## This would be passed into the algorithm.
assert options["type"] == "DataIntegrityProof"
assert options["cryptosuite"] == "schnorr-secp256k1-jcs-2024"

In [72]:
import jcs

In [73]:
canonical_document = jcs.canonicalize(unsecured_document)

In [74]:
print("Returning canonical document : ", canonical_document)
transformed_document= canonical_document

Returning canonical document :  b'{"@context":["https://www.w3.org/ns/credentials/v2","https://www.w3.org/ns/credentials/examples/v2"],"credentialSubject":{"alumniOf":{"id":"did:example:c276e12ec21ebfeb1f712ebc6f1","name":"Example University"},"id":"did:example:ebfeb1f712ebc6f1c276e12ec21"},"id":"http://university.example/credentials/58473","issuer":"did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65","type":["VerifiableCredential","ExampleAlumniCredential"],"validFrom":"2020-01-01T00:00:00Z"}'


## 5. Let hashData be the result of running the algorithm in Section XXX Hashing (schnorr-secp256k1-jcs-2024) with transformedData and proofConfig passed as a parameters.


Example Spec text: https://www.w3.org/TR/vc-di-eddsa/#hashing-eddsa-jcs-2022



### Spec Text

The following algorithm specifies how to cryptographically hash a transformed data document and proof configuration into cryptographic hash data that is ready to be provided as input to the algorithms in Section XXX Proof Serialization (schnorr-secp256k1-jcs-2024) or Section xxx Proof Verification (schnorr-secp256k1-jcs-2024).

The required inputs to this algorithm are a transformed data document (transformedDocument) and canonical proof configuration (canonicalProofConfig). A single hash data value represented as series of bytes is produced as output.

1. Let transformedDocumentHash be the result of applying the SHA-256 (SHA-2 with 256-bit output) cryptographic hashing algorithm [RFC6234] to the transformedDocument. transformedDocumentHash will be exactly 32 bytes in size.
2. Let proofConfigHash be the result of applying the SHA-256 (SHA-2 with 256-bit output) cryptographic hashing algorithm [RFC6234] to the canonicalProofConfig. proofConfigHash will be exactly 32 bytes in size.
3. Let hashData be the result of joining proofConfigHash (the first hash) with transformedDocumentHash (the second hash).
4. Return hashData as the hash data.

# Difference from EdDSA! We may have to concat the bytes of proofConfig and transformedDocument then hash the result

Or we could combine the hashes then hash the result. I believe with Schnorr as defined in BIP340 we need to have an input of 32 bytes.

## Also, this might just be a limitation of the library I am using.

### 5.1 Let transformedDocumentHash be the result of applying the SHA-256 (SHA-2 with 256-bit output) cryptographic hashing algorithm [RFC6234] to the transformedDocument. transformedDocumentHash will be exactly 32 bytes in size.

In [75]:
from buidl.helper import sha256

In [76]:

#transformed_document_hash = sha256(transformed_document)

In [77]:
#assert(len(transformed_document_hash) == 32)

### 5.2. Let proofConfigHash be the result of applying the SHA-256 (SHA-2 with 256-bit output) cryptographic hashing algorithm [RFC6234] to the canonicalProofConfig. proofConfigHash will be exactly 32 bytes in size.


In [78]:
#proof_config_hash = sha256(proof_config)
#assert(len(proof_config_hash) == 32)

In [79]:
#proof_config_hash

### 5.3. Let hashData be the result of joining proofConfigHash (the first hash) with transformedDocumentHash (the second hash).


In [80]:
# hash_data = proof_config_hash + transformed_document_hash

## Possible Updated Hash Impl

In [81]:
data_to_hash = proof_config + transformed_document
hash_data = sha256(data_to_hash)

### 5.4. Return hashData as the hash data.

In [82]:
print("Return hashData", hash_data)

Return hashData b'\xc9X<\xf9p\xb1C\xba\x84\x18 \x0fr$<U\xe9\\R\x13\xc1\xb1\xcf\x04\xe0L\xbf\x01\x06\xa9\xa4\t'


## 6. Let proofBytes be the result of running the algorithm in Section XXX Proof Serialization (schnorr-secp256k1-jcs-2024) with hashData and options passed as parameters.




### Spec Text

The following algorithm specifies how to serialize a digital signature from a set of cryptographic hash data. This algorithm is designed to be used in conjunction with the algorithms defined in the Data Integrity [VC-DATA-INTEGRITY] specification, Section 4: Algorithms. Required inputs are cryptographic hash data (hashData) and proof options (options). The proof options MUST contain a type identifier for the cryptographic suite (type) and MAY contain a cryptosuite identifier (cryptosuite). A single digital proof value represented as series of bytes is produced as output.

1. Let privateKeyBytes be the result of retrieving the private key bytes associated with the options.verificationMethod value as described in the Data Integrity [VC-DATA-INTEGRITY] specification, Section 4.1: Processing Model.
2. Let proofBytes be the result of applying the Schnorr Signature Alogirthm [BIP340], using the Secp256k1 variant, with hashData as the data to be signed using the private key specified by privateKeyBytes. proofBytes will be exactly 64 bytes in size.
3. Return proofBytes as the digital proof.


### 6.1 Let privateKeyBytes be the result of retrieving the private key bytes associated with the options.verificationMethod value as described in the Data Integrity [VC-DATA-INTEGRITY] specification, Section 4.1: Processing Model.

Note: Not fully certain about this step. Partly because in these notebooks I already have the private key in an object. I think likely it is something like as follows below.

In [83]:
from buidl.helper import int_to_big_endian

In [84]:
privateKeyBytes = int_to_big_endian(initial_sk.secret, 32)

In [85]:
privateKeyBytes

b's\xfd\xdc\x12\xfc\x93B\xbb)\xae\x9b^\xd4v2;\xdci:\x11n6Q$U\xae\xe80\xfe\x8a%\xa2'

### 6.2. Let proofBytes be the result of applying the Schnorr Signature Alogirthm [BIP340], using the Secp256k1 variant, with hashData as the data to be signed using the private key specified by privateKeyBytes. proofBytes will be exactly 64 bytes in size.

In [86]:
# TODO: We probably want to specify how aux data should be generated
# From BIP340 The auxiliary random data should be set to fresh randomness generated at signing time, resulting in what is called a synthetic nonce. Using 32 bytes of randomness is optimal.
aux=b"\x00" * 32

schnorr_sig = initial_sk.sign_schnorr(hash_data, aux)

proof_bytes = schnorr_sig.serialize()

### 6.3. Return proofBytes as the digital proof.

In [87]:
print("Return Proof Bytes : ", proof_bytes)

Return Proof Bytes :  b"+\xf0\xe6\xff\xe7\x95\xee\xbbb\x14\xf8\r\xa9\x88\x81\x82\x02@[\xb3\x98#j\xba\x7f\xf0Z\x9f&\x12\x01\xa3L\xd2\xc6\x92 L\x19G\x8d\xfb\xba\xc5\xa7\xfd\x8b\xbe\x16'n\xb9\t\xa1\xad,f+R\x1c\x96X\xf9\x08"


## 7. Let proof.proofValue be a base58-btc-encoded Multibase value of the proofBytes.

THIS IS A QUESTION. DO WE WANT TO USE base58btc. We do not want to use base58, this if for the stuff you are showing to other people and talk about it on the phone.

What is the most efficient representation for how this is passed around in computers.

Probably the best is base64 here.


In [90]:
from multiformats import multibase

proof_value = multibase.encode(proof_bytes, "base58btc")

proof["proofValue"] = proof_value

In [91]:
print("Return proof", json.dumps(proof, indent=2))

Return proof {
  "type": "DataIntegrityProof",
  "cryptosuite": "schnorr-secp256k1-jcs-2024",
  "verificationMethod": "did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65#initialKey",
  "proofPurpose": "assertionMethod",
  "@context": [
    "https://www.w3.org/ns/credentials/v2",
    "https://www.w3.org/ns/credentials/examples/v2"
  ],
  "proofValue": "zsxM9je5iKynKyN6rRPi9QjTWpG6inJ1umwGfnCo4fiu4MqYf46PLd4TE2wVZvdZegDuC6xL6n3Kj8S1PbC8tmTm"
}


## 8. Let securedDataDocument be a copy of inputDocument. (Add Proof)

In [92]:
secured_data_document = copy.deepcopy(unsecured_document)

## 9. Set securedDataDocument.proof to the value of proof. (Add Proof)


In [93]:
secured_data_document["proof"] = proof

## 10. Return securedDataDocument as the secured data document.

In [95]:
print("Return securedDataDocument")
print(json.dumps(secured_data_document, indent=2))

Return securedDataDocument
{
  "@context": [
    "https://www.w3.org/ns/credentials/v2",
    "https://www.w3.org/ns/credentials/examples/v2"
  ],
  "id": "http://university.example/credentials/58473",
  "type": [
    "VerifiableCredential",
    "ExampleAlumniCredential"
  ],
  "validFrom": "2020-01-01T00:00:00Z",
  "credentialSubject": {
    "id": "did:example:ebfeb1f712ebc6f1c276e12ec21",
    "alumniOf": {
      "id": "did:example:c276e12ec21ebfeb1f712ebc6f1",
      "name": "Example University"
    }
  },
  "issuer": "did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65",
  "proof": {
    "type": "DataIntegrityProof",
    "cryptosuite": "schnorr-secp256k1-jcs-2024",
    "verificationMethod": "did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65#initialKey",
    "proofPurpose": "assertionMethod",
    "@context": [
      "https://www.w3.org/ns/credentials/v2",
      "https://www.w3.org/ns/credentials/examples/v2"
    ],
    "proofValue": "zsxM9je5iKyn

## Copy the Secured VC into the VC verification notebook to verify it


In [96]:
print(f"secured_vc = {secured_data_document}")

secured_vc = {'@context': ['https://www.w3.org/ns/credentials/v2', 'https://www.w3.org/ns/credentials/examples/v2'], 'id': 'http://university.example/credentials/58473', 'type': ['VerifiableCredential', 'ExampleAlumniCredential'], 'validFrom': '2020-01-01T00:00:00Z', 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21', 'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1', 'name': 'Example University'}}, 'issuer': 'did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65', 'proof': {'type': 'DataIntegrityProof', 'cryptosuite': 'schnorr-secp256k1-jcs-2024', 'verificationMethod': 'did:btc1:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65#initialKey', 'proofPurpose': 'assertionMethod', '@context': ['https://www.w3.org/ns/credentials/v2', 'https://www.w3.org/ns/credentials/examples/v2'], 'proofValue': 'zsxM9je5iKynKyN6rRPi9QjTWpG6inJ1umwGfnCo4fiu4MqYf46PLd4TE2wVZvdZegDuC6xL6n3Kj8S1PbC8tmTm'}}
